In [ ]:
import os
import math
import nglview
from ipywidgets import widgets

from moleidoscope.linker import Linker, view_linkers
from moleidoscope.geo.coor import Coor
from moleidoscope.mirror import Mirror
from moleidoscope.line import Line

## Display Linkers ##

In [ ]:
# 1 - Display linker
link1 = Linker(21)
link1.view()

## Translation ##

In [ ]:
# 2 - Translate linker, display multiple linkers
link1 = Linker(21)

link2 = Linker(21)
 # linker.translate([x, y, z])
def f(x):
    print('Translating %i ...' %x)
    link2.translate([x, 0, 0])
    return view_linkers(link1, link2)

widgets.interact(f, x=(0., 20., 2))

## Reflection in common planes ##

In [ ]:
link = Linker(142)                                   # Create a new linker from library
link.center(coor=[5, 5, 5])                          # Center the linker to [5, 5, 5]

link2 = link.reflect('xy', translate=None)           # Reflect along XY plane
link3 = link.reflect('xz', translate=None)           # Reflect along XZ plane
link4 = link.reflect('yz', translate=None)           # Reflect along YZ plane

view_linkers(link, link2, link3, link4,
             Line.xyz(size=20),                      # xyz axis
             Mirror('xy').to_linker(),               # XY plane -> blue
             Mirror('yz').to_linker(atom_type='O'),  # YZ plane -> red
             Mirror('xz').to_linker(atom_type='C'))  # XZ plane -> gray

## Reflection in arbitrary planes

In [ ]:
# 1 Reflection in common planes

link = Linker(142)
link.translate([5, 3, 5])

# Define a plane in 3D space for reflection
p1 = [1, 2, 6]
p2 = [10, 4, -9]
p3 = [-0.1, 4, 0]
mir = Mirror(p1, p2, p3, size=1)
#link.center(mirror=mir)

link2 = link.reflect(mir)
view_linkers(link, link2, mir.to_linker())

## Rotation

In [ ]:
# ROTATION ---------------------------------------------------------

# Determine rotation -> [angle, p1, p2] where p1----p2 is the rotation axis
rotation_info = [math.pi, [0, 0, 0], [1, 0, 0]]

# Rotate l1
l4 = l1.rotate(rotation_info)
l4.translate([0, 0, -6])

# Join rotated and original linkers
l5 = l4.join(l1)
l5.view()

## Rotoreflection (Improper Rotation)

In [ ]:
# ROTOREFLECTION (Improper Rotation) ---------------------------------------------

# Determine rotation -> [angle, p1, p2] where p1----p2 is the rotation axis
rotation_info = [math.pi, [0, 0, 0], [1, 0, 0]]
# Determine reflection plane as 'xy

lr = l1.rotoreflect(rotation_info, 'xy')
lr.translate([0, 0, -7])

# Join rotoreflection and original linker and plane
lr = lr.join(l1)

lr.view()